In [1]:
# %pip install --upgrade transformers
# %pip install --upgrade torch


In [5]:
%conda activate mca
%pip install protobuf==3.20.3
%pip install hf_xet
%pip install ipywidgets



Note: you may need to restart the kernel to use updated packages.


'C:\Users\SUJAL' is not recognized as an internal or external command,
operable program or batch file.


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os

In [4]:
API_KEY = os.getenv('GROQ_API_KEY')


In [ ]:
import os
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
import requests
import torch

# ------------------------------
# 1. Load Dataset
# ------------------------------
df = pd.read_csv("combined_comments.csv")
df['Comment'] = df['Comment'].fillna('')

# ------------------------------
# 2. Check for GPU
# ------------------------------
device = 0 if torch.cuda.is_available() else -1
print(f"Device set to: {'GPU' if device==0 else 'CPU'}")

# ------------------------------
# 3. Load ABSA Model
# ------------------------------
print("🔄 Loading ABSA model...")
absa = pipeline(
    "text-classification",
    model="yangheng/deberta-v3-base-absa-v1.1",
    tokenizer="yangheng/deberta-v3-base-absa-v1.1",
    device=device
)

# ------------------------------
# 4. Run ABSA in batches for speed
# ------------------------------
print("🔄 Running ABSA on comments...")

batch_size = 64  # increase if GPU memory allows
results = []

for i in tqdm(range(0, len(df), batch_size), desc="Processing ABSA"):
    batch_texts = df['Comment'].iloc[i:i+batch_size].tolist()
    try:
        batch_output = absa(batch_texts, truncation=True, max_length=512)
        results.extend([o['label'] for o in batch_output])
    except Exception as e:
        results.extend(["Error"] * len(batch_texts))

df['ABSA_Sentiment'] = results

# Save ABSA results
df.to_csv("absa_results.csv", index=False)
print("✅ ABSA completed. Results saved in absa_results.csv")

# ------------------------------
# 5. Prepare Summary Stats
# ------------------------------
positive_count = sum(df['ABSA_Sentiment'] == "Positive")
negative_count = sum(df['ABSA_Sentiment'] == "Negative")
neutral_count = sum(df['ABSA_Sentiment'] == "Neutral")

summary_input = (
    f"Out of {len(df)} comments:\n"
    f"Positive: {positive_count}\n"
    f"Negative: {negative_count}\n"
    f"Neutral: {neutral_count}\n"
)

print("\n📊 Sentiment Distribution:")
print(summary_input)

# ------------------------------
# 6. Send to Groq API for Summarization
# ------------------------------
API_URL = "https://api.groq.com/openai/v1/chat/completions"

headers = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}
payload = {
    "model": "mixtral-8x7b-32768",
    "messages": [
        {"role": "system", "content": "You are an assistant that summarizes sentiment analysis results."},
        {"role": "user", "content": f"Summarize these sentiment counts in a short paragraph:\n{summary_input}"}
    ],
    "temperature": 0.3,
}

try:
    response = requests.post(API_URL, headers=headers, json=payload)
    response.raise_for_status()
    result = response.json()
    final_summary = result['choices'][0]['message']['content']
    print("\n📝 Groq Summary:")
    print(final_summary)
except Exception as e:
    print(f"⚠️ Groq API Error: {e}")


Device set to: GPU
🔄 Loading ABSA model...


c:\Users\SUJAL DAS\anaconda3\envs\mca\lib\site-packages\transformers\convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Device set to use cuda:0


🔄 Running ABSA on comments...


Processing ABSA: 100%|██████████| 450/450 [17:54<00:00,  2.39s/it]


✅ ABSA completed. Results saved in absa_results.csv

📊 Sentiment Distribution:
Out of 28756 comments:
Positive: 10152
Negative: 8663
Neutral: 9941

⚠️ Groq API Error: 400 Client Error: Bad Request for url: https://api.groq.com/openai/v1/chat/completions
